# Heat Cython Serial

Cython Serial Implementation of the Test Problem

In [2]:
! cython --version

Cython version 0.29.20


In [2]:
! ifort --version

ifort (IFORT) 19.0.3.199 20190206
Copyright (C) 1985-2019 Intel Corporation.  All rights reserved.



In [3]:
! icc --version

icc (ICC) 19.0.3.199 20190206
Copyright (C) 1985-2019 Intel Corporation.  All rights reserved.



### Cython module

In [6]:
%%writefile scs.pyx
#cython: boundscheck=False, wraparound=False, cdivision=True
#cython: initializedcheck=False, language_level=3, infer_types=True

cpdef st(int n, double energy, int niters):
    from time import time
    import numpy as np

    # definição de variáveis
    cdef double      heat      = 0.0
    cdef double      t         = 0.0
    cdef Py_ssize_t  size      = n + 2
    cdef Py_ssize_t  sizeStart = 1
    cdef Py_ssize_t  sizeEnd   = n + 1
    cdef Py_ssize_t  iters, i, j

    t = time()
    
    # cria e inicializa as matrizes com zeros e memoryview
    cdef double[:,::1] mvaold = np.zeros((size, size), np.double)
    cdef double[:,::1] mvanew = np.zeros((size, size), np.double)
    cdef Py_ssize_t    nsources  = 3      # qde de fontes
    cdef    int[:,::1] mvsources = np.empty( (nsources,2), np.intc)

    # inicializa 3 fontes de calor
    mvsources[0,0] = mvsources[0,1] = n/2
    mvsources[1,0] = mvsources[1,1] = n/3
    mvsources[2,0] = n*4/5
    mvsources[2,1] = n*8/9

    niters = (niters + 1) // 2
    for iters in range(niters) :
        # iteracao impar
        for i in range(sizeStart, sizeEnd) :
            for j in range(sizeStart, sizeEnd) :
                mvanew[i,j]=1/2.0*(mvaold[i,j]+1/4.0*(mvaold[i-1,j]+mvaold[i+1,j]+mvaold[i,j-1]+mvaold[i,j+1]))
        for i in range(nsources) :
            mvanew[mvsources[i,0], mvsources[i,1]] += energy
        # iteracao par
        for i in range(sizeStart, sizeEnd) :
            for j in range(sizeStart, sizeEnd) :
                mvaold[i,j]=1/2.0*(mvanew[i,j]+1/4.0*(mvanew[i-1,j]+mvanew[i+1,j]+mvanew[i,j-1]+mvanew[i,j+1] ) / 8.)
        for i in range(nsources) :
            mvaold[mvsources[i,0], mvsources[i,1]] += energy
    # calcula o total de energia
    for i in range(sizeStart, sizeEnd) :
        for j in range(sizeStart, sizeEnd) :
            heat += mvaold[i,j]
    t = time() - t
#    print("Heat = %0.4f | Tempo = %0.4f" %(heat, t))
    return heat, t

Overwriting scs.pyx


### Setup file

In [7]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("scs.pyx", force=True)
)

Overwriting setup.py


### Main

In [9]:
%%writefile st-cy-seq.py
from time import time
tp = time()
import scs

n            = 4800    # nxn grid; 4800,1,500→1500; 100,1,10→30 [4800]
energy       = 1.0     # energy to be injected per iteration [1.0]
niters       = 500     # number of iterations [500]

heat, t = scs.st(n, energy, niters)
tp = time() - tp
print("Heat = %0.4f | Tempo = %0.4f | TempoPyt = %0.4f" %(heat, t, tp))

Overwriting st-cy-seq.py


### Build

In [11]:
%%bash
rm scs.*.so
python setup.py build_ext --inplace

[1/1] Cythonizing scs.pyx
running build_ext
building 'scs' extension
gcc -pthread -B /scratch/app/anaconda3/2018.12/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/scratch/app/anaconda3/2018.12/include/python3.7m -c scs.c -o build/temp.linux-x86_64-3.7/scs.o
gcc -pthread -shared -B /scratch/app/anaconda3/2018.12/compiler_compat -L/scratch/app/anaconda3/2018.12/lib -Wl,-rpath=/scratch/app/anaconda3/2018.12/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.7/scs.o -o build/lib.linux-x86_64-3.7/scs.cpython-37m-x86_64-linux-gnu.so
copying build/lib.linux-x86_64-3.7/scs.cpython-37m-x86_64-linux-gnu.so -> 


### Check

In [12]:
%%bash
python st-cy-seq.py

Heat = 1500.0000 | Tempo = 19.2412 | TempoPyt = 19.4514


### Running on an execution node

Copy the executable to /scratch :

In [17]:
%%bash
s='/prj/yyyy/xxxx/stnc/Cython'
d='/scratch/yyyy/xxxx/stnc/Cython'
rm $d/scs.*.so
cp  $s/scs.*.so  $s/st-cy-seq.py  $s/calctempo.sh  $d

Checking if it works, before batch execution

In [18]:
%%bash
a='/scratch/yyyy/xxxx'
d='/scratch/yyyy/xxxx/stnc/Cython'
cd $d
sh calctempo.sh

Heat = 1500.0000 | Tempo = 21.1068 | TempoPyt = 21.3335



real	0m21.393s
user	0m20.228s
sys	0m1.126s


### Slurm script

In [19]:
%%writefile st-cy-seq.srm
#!/bin/bash
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH --ntasks=1             #Total de tarefas
#SBATCH --nodes=1              #Qtd de nós
#SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
#SBATCH -J stcyseq             #Nome do job, 8 caracteres
#SBATCH --exclusive            #Utilização exclusiva dos nós
#SBATCH --time=00:02:00        #Tempo max. de execução 2 minutos

echo '========================================'
echo '- Stencil Cython Sequencial'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
cd /scratch/yyyy/xxxx/stnc/Cython

#Executavel
EXEC='sh calctempo.sh'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ time srun -n ' $SLURM_NTASKS $EXEC
srun -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting st-cy-seq.srm


### Sends to run 3 times

In [20]:
%%bash
sbatch st-cy-seq.srm
sbatch st-cy-seq.srm
sbatch st-cy-seq.srm
squeue -n stcyseq

Submitted batch job 772612
Submitted batch job 772613
Submitted batch job 772614
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772612 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)
            772613 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)
            772614 cpu_small  stcyseq xxxx. PD       0:00      1 (Priority)


Checks if it is finished

In [25]:
%%bash
squeue -n stcyseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772614 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)
            772613 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)
            772612 cpu_small  stcyseq xxxx. PD       0:00      1 (Resources)


#### Shows the output files

In [1]:
%%bash
b='/stnc/Cython'
d='/scratch/yyyy/xxxx'$b
cat $d/slurm-772612.out
cat $d/slurm-772613.out
cat $d/slurm-772614.out

- Stencil Cython Sequencial
- Job ID: 772612
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1407
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/yyyy/xxxx/stnc/Cython
-- srun -------------------------------
$ time srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 23.9847 | TempoPyt = 29.4011

real	0m29.715s
user	0m24.100s
sys	0m0.335s
-- FIM --------------------------------
- Stencil Cython Sequencial
- Job ID: 772613
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1407
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/yyyy/xxxx/stnc/Cython
-- srun -------------------------------
$ time srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 23.9444 | TempoPyt = 24.6716

real	0m24.811s
user	0m24.077s
sys	0m0.255s
-- FIM --------------------------------
- Stencil Cython Sequencial
- Job ID: 772614
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1407
- diretorio